Notebook purpose

- Sense-check and explore calculated salaries

In [14]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl
import entropy.data.creators as cr

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
SAMPLE = '777'
fp = f'~/tmp/entropy_{SAMPLE}.parquet'

df = aws.read_parquet(fp)
df.head(1)

,id,date,user_id,amount,desc,merchant,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,ym,balance
0,688261,2012-01-03,777,400.0,<mdbremoved> - s/o,NaN,other account,False,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,2014-07-18,2017-11-13,True,364.220001,non merchant mbl,transfers,other account,other account,201201,2179.515625


food, groceries, household    16604
transfers                     15742
enjoyment                      9072
cash                           7712
lunch or snacks                5298
bank charges                   4151
public transport               3870
fuel                           3390
supermarket                    2938
dining or going out            2733
dining and drinking            2696
groceries                      2581
mobile                         2300
credit card                    2184
current account                2136
entertainment, tv, media       1956
earnings_income                1833
clothes - everyday or work     1827
gambling                       1747
taxi                           1496
Name: tag, dtype: int64

In [19]:
earnings = [
    'salary or wages \(main\)',
    'salary or wages \(other\)',
    'salary \(secondary\)',
]

pat = '|'.join(earnings)
df[df.tag_up.str.match(pat, na=False) & ~df.debit].tag_up.value_counts().sum()


1833

In [7]:
df.tag.str.match('salary and wages \(main\)').sum()

0

In [32]:
type(r'ko')

str

In [66]:
df[df.tag.str.contains('invest', na=False)]

,id,date,user_id,amount,desc,merchant,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,ym,balance
7259,6037628,2013-04-30,22777,1.00,<mdbremoved>,NaN,investment - other,True,cb4 3,2013-05-28,NaN,1980.0,2013-05-28,4149,2015-01-03 06:52:00,hsbc,current,2014-07-18,2017-11-13,True,-852.200012,NaN,transfers,investment - other,investment - other,201304,-5481.270508
7833,33823757,2014-08-01,22777,5000.00,<mdbremoved> ltd <mdbremoved>,NaN,investment - other,True,cb4 3,2013-05-28,NaN,1980.0,2013-05-28,4149,2015-01-03 06:52:00,hsbc,current,2014-09-06,2017-11-13,True,-852.200012,NaN,NaN,investment - other,investment - other,201408,-1073.630859
15667,178957058,2016-12-28,53777,36.00,"greyfriars llp greyfriars llp , initial payment - don't recognise this retailer? type: direct deb",greyfriars llp,pension or investments,True,bs4 4,2014-05-10,NaN,1989.0,2014-05-10,56724,2017-11-28 16:21:00,natwest bank,current,2017-01-03,2018-04-30,True,NaN,greyfriars llp,pension or investments,NaN,pension or investments,201612,NaN
15734,180227950,2017-01-03,53777,2.00,greyfriars llp type: direct debit - d/d,greyfriars llp,pension or investments,True,bs4 4,2014-05-10,NaN,1989.0,2014-05-10,56724,2017-11-28 16:21:00,natwest bank,current,2017-01-09,2018-04-30,True,NaN,greyfriars llp,pension or investments,NaN,pension or investments,201701,NaN
15823,183622527,2017-01-23,53777,9.00,greyfriars llp type: direct debit - d/d,greyfriars llp,pension or investments,True,bs4 4,2014-05-10,NaN,1989.0,2014-05-10,56724,2017-11-28 16:21:00,natwest bank,current,2017-01-29,2018-04-30,True,NaN,greyfriars llp,pension or investments,NaN,pension or investments,201701,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153884,758829273,2020-04-15,573777,10.00,family equity plan,family equity plan ltd,investment - other,False,bt66 6,2020-02-24,NaN,1973.0,2020-02-24,1608612,2020-05-23 23:43:00,halifax personal banking,current,2020-04-16,1900-01-01,True,53.430000,family equity plan ltd,investment - other,NaN,investment - other,202004,-800.529663
153885,758829274,2020-04-15,573777,10.00,family equity plan,family equity plan ltd,investment - other,False,bt66 6,2020-02-24,NaN,1973.0,2020-02-24,1608612,2020-05-23 23:43:00,halifax personal banking,current,2020-04-16,1900-01-01,True,53.430000,family equity plan ltd,investment - other,NaN,investment - other,202004,-800.529663
153958,773244817,2020-05-15,573777,10.00,family equity plan,family equity plan ltd,investment - other,False,bt66 6,2020-02-24,NaN,1973.0,2020-02-24,1608612,2020-05-23 23:43:00,halifax personal banking,current,2020-05-16,1900-01-01,True,53.430000,family equity plan ltd,investment - other,NaN,investment - other,202005,-502.530029
153959,773244818,2020-05-15,573777,10.00,family equity plan,family equity plan ltd,investment - other,False,bt66 6,2020-02-24,NaN,1973.0,2020-02-24,1608612,2020-05-23 23:43:00,halifax personal banking,current,2020-05-16,1900-01-01,True,53.430000,family equity plan ltd,investment - other,NaN,investment - other,202005,-502.530029
